In [283]:
import pandas as pd
import functools as fn 
import numpy as np

In [2]:
!git clone https://github.com/nghi-huynh/BigDataChallenge2022.git

Cloning into 'BigDataChallenge2022'...
remote: Enumerating objects: 72, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 72 (delta 22), reused 57 (delta 10), pack-reused 0
Unpacking objects: 100% (72/72), done.


In [3]:
!mv /content/BigDataChallenge2022/raw_data .
!mv /content/BigDataChallenge2022/merged_data .
!rm -r /content/BigDataChallenge2022

## Helper functions

In [4]:
# melt dataframes based on id columns
def melt(df,id_vars, var_name, value_name):
  tmp = df.melt(id_vars=id_vars,var_name=var_name,value_name=value_name)
  return tmp

In [5]:
# drop columns and melt dataframe 
def preprocessing(df, cols_drop, melt_vars):
  tmp = df.drop(cols_drop, axis=1)
  tmp = melt(tmp, melt_vars[0], melt_vars[1], melt_vars[2])
  return tmp

In [21]:
# merge data frames
# specify left on, right on, and how
def merge_data(dataframes, left_on, right_on, how):
  tmp = fn.reduce(lambda  left,right: pd.merge(left,right,left_on=left_on,
                                                   right_on=right_on,
                                                   how=how), dataframes)
                                            
  return tmp

Since our raw data have two types of structure, I want to separate them into 2 parts and merge them based on their current structure first. Then, I re-structured the merged dataframes and merged all of them together.

## Merge deaths related to mental disorders, suicide rate

### Load and check

In [341]:
mental_substance_death = pd.read_csv("/content/raw_data/death-rates-from-mental-and-substance-disorders-by-age.csv")

In [342]:
mental_substance_death.columns

Index(['Entity', 'Code', 'Year',
       'Deaths - Mental and substance use disorders - Sex: Both - Age: Under 5 (Rate)',
       'Deaths - Mental and substance use disorders - Sex: Both - Age: 70+ years (Rate)',
       'Deaths - Mental and substance use disorders - Sex: Both - Age: 5-14 years (Rate)',
       'Deaths - Mental and substance use disorders - Sex: Both - Age: 15-49 years (Rate)',
       'Deaths - Mental and substance use disorders - Sex: Both - Age: 50-69 years (Rate)',
       'Deaths - Mental and substance use disorders - Sex: Both - Age: Age-standardized (Rate)',
       'Deaths - Mental and substance use disorders - Sex: Both - Age: All Ages (Rate)'],
      dtype='object')

In [343]:
mental_substance_death.isnull().sum()

Entity                                                                                      0
Code                                                                                      980
Year                                                                                        0
Deaths - Mental and substance use disorders - Sex: Both - Age: Under 5 (Rate)             312
Deaths - Mental and substance use disorders - Sex: Both - Age: 70+ years (Rate)           312
Deaths - Mental and substance use disorders - Sex: Both - Age: 5-14 years (Rate)            0
Deaths - Mental and substance use disorders - Sex: Both - Age: 15-49 years (Rate)           0
Deaths - Mental and substance use disorders - Sex: Both - Age: 50-69 years (Rate)         312
Deaths - Mental and substance use disorders - Sex: Both - Age: Age-standardized (Rate)      0
Deaths - Mental and substance use disorders - Sex: Both - Age: All Ages (Rate)              0
dtype: int64

In [24]:
# Select columns we want to keep
cols = ['Entity','Year','Deaths - Mental and substance use disorders - Sex: Both - Age: Age-standardized (Rate)']

In [25]:
mental_substance_death_final = mental_substance_death[cols]

In [26]:
mental_substance_death_final

,Entity,Year,Deaths - Mental and substance use disorders - Sex: Both - Age: Age-standardized (Rate)
0,Afghanistan,1990,0.144067
1,Afghanistan,1991,0.140653
2,Afghanistan,1992,0.135279
3,Afghanistan,1993,0.133047
4,Afghanistan,1994,0.130073
...,...,...,...
6463,Zimbabwe,2013,1.379243
6464,Zimbabwe,2014,1.398058
6465,Zimbabwe,2015,1.413464
6466,Zimbabwe,2016,1.445058


In [47]:
mental_substance_death_final.isnull().sum()

Entity                                                                                    0
Year                                                                                      0
Deaths - Mental and substance use disorders - Sex: Both - Age: Age-standardized (Rate)    0
dtype: int64

In [85]:
countries_a = list(mental_substance_death_final['Entity'].unique())
len(countries_a)

231

In [56]:
self_harm_death = pd.read_csv("/content/raw_data/share-deaths-suicide.csv")
self_harm_death = self_harm_death.drop(["Code"], axis=1)
self_harm_death

,Entity,Year,Deaths - Self-harm - Sex: Both - Age: All Ages (Percent)
0,Afghanistan,1990,0.381465
1,Afghanistan,1991,0.388646
2,Afghanistan,1992,0.409441
3,Afghanistan,1993,0.418030
4,Afghanistan,1994,0.409779
...,...,...,...
8095,Zimbabwe,2015,1.676438
8096,Zimbabwe,2016,1.747906
8097,Zimbabwe,2017,1.804474
8098,Zimbabwe,2018,1.870430


In [84]:
countries_b = list(self_harm_death['Entity'].unique())
len(countries_b)

270

In [58]:
self_harm_death.isnull().sum()

Entity                                                      0
Year                                                        0
Deaths - Self-harm - Sex: Both - Age: All Ages (Percent)    0
dtype: int64

In [57]:
mental_death = pd.read_csv("/content/raw_data/share-with-mental-and-substance-disorders.csv")
mental_death = mental_death.drop(["Code"], axis=1)
mental_death

,Entity,Year,Prevalence - Mental disorders - Sex: Both - Age: Age-standardized (Percent)
0,Afghanistan,1990,16.659229
1,Afghanistan,1991,16.765052
2,Afghanistan,1992,16.874469
3,Afghanistan,1993,16.990720
4,Afghanistan,1994,17.112730
...,...,...,...
6895,Zimbabwe,2015,11.156429
6896,Zimbabwe,2016,11.164133
6897,Zimbabwe,2017,11.170427
6898,Zimbabwe,2018,11.158765


In [59]:
mental_death.isnull().sum()

Entity                                                                         0
Year                                                                           0
Prevalence - Mental disorders - Sex: Both - Age: Age-standardized (Percent)    0
dtype: int64

In [92]:
countries_c = mental_death['Entity'].unique()
len(countries_c)

230

### Suicide rate

In [256]:
suicide_rate = pd.read_csv("/content/raw_data/suicide_rate_country_age_standardized.csv")

In [257]:
# filter only both sexes 
suicide_both_sexes = suicide_rate.loc[suicide_rate['Sex'] == 'Both sexes',:].drop(['Sex'], axis=1)

In [258]:
suicide_both_sexes.head()

,Country,2019,2018,2017,2016,2015,2014,2013,2012,2011,...,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000
0,Afghanistan,6.0 [3.4-9.9],5.9 [3.4-9.8],6.0 [3.4-9.9],6.0 [3.5-9.9],6.0 [3.5-9.9],6.0 [3.5-9.9],6.2 [3.6-10.2],6.2 [3.6-10.2],6.4 [3.8-10.5],...,6.8 [4.0-11.1],7.2 [4.2-11.7],7.4 [4.3-12.1],7.6 [4.4-12.3],7.6 [4.4-12.3],7.8 [4.5-12.6],7.7 [4.5-12.5],7.9 [4.5-12.8],7.9 [4.5-12.8],7.7 [4.4-12.5]
3,Albania,3.7 [2.1-5.7],3.9 [2.2-6.0],4.1 [2.3-6.3],4.2 [2.3-6.3],4.2 [2.3-6.4],4.5 [2.5-6.7],4.8 [2.7-7.1],4.8 [2.8-7.0],7.6 [4.7-10.7],...,8.0 [4.8-10.8],8.1 [5.0-10.8],8.1 [5.0-10.6],7.8 [5.0-10.2],7.7 [4.9-10.0],4.9 [3.1-6.3],4.9 [3.3-6.4],4.8 [3.2-6.3],4.7 [3.2-6.2],5.2 [3.6-6.9]
6,Algeria,2.6 [1.4-4.4],2.6 [1.4-4.4],2.5 [1.4-4.4],2.6 [1.4-4.5],2.7 [1.5-4.7],2.8 [1.5-4.8],2.9 [1.5-5.0],2.9 [1.6-5.0],2.9 [1.6-5.1],...,3.2 [1.7-5.6],3.3 [1.8-5.8],3.5 [1.8-6.0],3.7 [2.0-6.4],3.8 [2.0-6.6],4.0 [2.1-6.9],4.1 [2.2-7.1],4.4 [2.4-7.6],4.6 [2.4-7.9],4.7 [2.5-8.0]
9,Angola,12.6 [7.7-19.4],12.4 [7.6-19.0],12.4 [7.6-18.9],12.9 [8.0-19.6],13.3 [8.3-20.1],13.1 [8.3-19.8],14.2 [9.0-21.2],14.2 [9.0-21.2],13.5 [8.6-20.2],...,13.6 [8.7-20.5],15.3 [9.7-23.0],15.0 [9.5-22.6],17.1 [10.8-25.6],16.3 [10.3-24.4],17.2 [10.8-25.9],17.5 [10.8-26.6],17.2 [10.7-26.3],17.5 [10.6-26.7],17.6 [10.5-27.0]
12,Antigua and Barbuda,0.3 [0.2-0.5],0.3 [0.2-0.5],0.0 [0.0-0.0],0.5 [0.3-0.7],0.4 [0.3-0.6],0.4 [0.3-0.6],0.0 [0.0-0.0],0.0 [0.0-0.0],0.2 [0.1-0.2],...,0.0 [0.0-0.0],0.2 [0.2-0.3],0.3 [0.2-0.5],0.4 [0.3-0.6],1.3 [0.9-1.8],2.0 [1.4-2.7],1.4 [1.0-2.0],1.3 [0.9-1.7],1.9 [1.3-2.6],2.0 [1.4-2.7]


In [259]:
cols_drop = []
melts_vars = [["Country"], "Year", "Suicide rate"]

In [260]:
suicide_both_sexes_final = preprocessing(suicide_both_sexes, cols_drop, melts_vars)

In [261]:
suicide_both_sexes_final.loc[suicide_both_sexes_final['Country'] == 'Albania',:]

,Country,Year,Suicide rate
1,Albania,2019,3.7 [2.1-5.7]
184,Albania,2018,3.9 [2.2-6.0]
367,Albania,2017,4.1 [2.3-6.3]
550,Albania,2016,4.2 [2.3-6.3]
733,Albania,2015,4.2 [2.3-6.4]
916,Albania,2014,4.5 [2.5-6.7]
1099,Albania,2013,4.8 [2.7-7.1]
1282,Albania,2012,4.8 [2.8-7.0]
1465,Albania,2011,7.6 [4.7-10.7]
1648,Albania,2010,7.6 [4.6-10.5]


In [262]:
suicide_both_sexes_final["Year"] = suicide_both_sexes_final["Year"].astype("int64")

In [263]:
suicide_both_sexes_final.dtypes

Country         object
Year             int64
Suicide rate    object
dtype: object

In [264]:
suicide_both_sexes_final.isnull().sum()

Country         0
Year            0
Suicide rate    0
dtype: int64

In [265]:
len(suicide_both_sexes_final['Country'].unique())

183

In [266]:
countries_suicide = suicide_both_sexes_final['Country'].unique()

In [274]:
suicide_both_sexes_final.rename(columns={'Country': 'Entity'}, inplace=True)

### Clean

In [267]:
# only select subset in the available countries from the referenced data
self_harm_death_final = self_harm_death.loc[self_harm_death['Entity'].isin(countries_suicide)]

In [268]:
len(self_harm_death_final['Entity'].unique())

164

In [269]:
self_harm_death_final.isnull().sum()

Entity                                                      0
Year                                                        0
Deaths - Self-harm - Sex: Both - Age: All Ages (Percent)    0
dtype: int64

In [271]:
mental_substance_final = mental_substance_death_final.loc[mental_substance_death_final['Entity'].isin(countries_suicide)]
print(len(mental_substance_final['Entity'].unique()))
mental_substance_final.isnull().sum()

164


Entity                                                                                    0
Year                                                                                      0
Deaths - Mental and substance use disorders - Sex: Both - Age: Age-standardized (Rate)    0
dtype: int64

In [272]:
mental_death_final = mental_death.loc[mental_death['Entity'].isin(countries_suicide)]
print(len(mental_death_final['Entity'].unique()))
mental_death_final.isnull().sum()

163


Entity                                                                         0
Year                                                                           0
Prevalence - Mental disorders - Sex: Both - Age: Age-standardized (Percent)    0
dtype: int64

### First Merge

**=>** Now, we have a set of dataframes with similar stucture to merge.

In [275]:
# compile the list of dataframes we want to merge
data_frames = [mental_death_final,mental_substance_final, self_harm_death_final, suicide_both_sexes_final]
left_on = ['Entity', 'Year']
right_on = ['Entity', 'Year']
how = 'left'

In [276]:
first_merged = merge_data(data_frames, left_on, right_on, how)

In [277]:
first_merged

,Entity,Year,Prevalence - Mental disorders - Sex: Both - Age: Age-standardized (Percent),Deaths - Mental and substance use disorders - Sex: Both - Age: Age-standardized (Rate),Deaths - Self-harm - Sex: Both - Age: All Ages (Percent),Suicide rate
0,Afghanistan,1990,16.659229,0.144067,0.381465,NaN
1,Afghanistan,1991,16.765052,0.140653,0.388646,NaN
2,Afghanistan,1992,16.874469,0.135279,0.409441,NaN
3,Afghanistan,1993,16.990720,0.133047,0.418030,NaN
4,Afghanistan,1994,17.112730,0.130073,0.409779,NaN
...,...,...,...,...,...,...
4885,Zimbabwe,2015,11.156429,1.413464,1.676438,30.7 [16.6-48.3]
4886,Zimbabwe,2016,11.164133,1.445058,1.747906,28.7 [15.4-45.3]
4887,Zimbabwe,2017,11.170427,1.470020,1.804474,25.9 [13.8-40.9]
4888,Zimbabwe,2018,11.158765,NaN,1.870430,23.9 [12.7-37.8]


In [278]:
first_merged.dtypes

Entity                                                                                     object
Year                                                                                        int64
Prevalence - Mental disorders - Sex: Both - Age: Age-standardized (Percent)               float64
Deaths - Mental and substance use disorders - Sex: Both - Age: Age-standardized (Rate)    float64
Deaths - Self-harm - Sex: Both - Age: All Ages (Percent)                                  float64
Suicide rate                                                                               object
dtype: object

In [279]:
len(first_merged['Entity'].unique())

163

In [284]:
years_keep = list(np.arange(2000,2020))

In [285]:
first_merged = first_merged.loc[first_merged['Year'].isin(years_keep)]

In [286]:
first_merged.isnull().sum()

Entity                                                                                      0
Year                                                                                        0
Prevalence - Mental disorders - Sex: Both - Age: Age-standardized (Percent)                 0
Deaths - Mental and substance use disorders - Sex: Both - Age: Age-standardized (Rate)    326
Deaths - Self-harm - Sex: Both - Age: All Ages (Percent)                                    0
Suicide rate                                                                                0
dtype: int64

In [287]:
# Save first merged to csv file
first_merged.to_csv('./first_merge_data.csv', index=False)

## Merge gdp, unemployment rate, and income

### GDP (current, per capita)

In [288]:
gdp_current = pd.read_csv("/content/raw_data/gdp_current.csv")

In [289]:

cols_drop = ['Country Code', 'Indicator Name', 'Indicator Code']
melts_var = [["Country Name"], "Year", "gdp_current"]

In [290]:
gdp_current_final = preprocessing(gdp_current, cols_drop, melts_var)

In [291]:
gdp_current_final

,Country Name,Year,gdp_current
0,Aruba,1960,NaN
1,Africa Eastern and Southern,1960,2.008272e+10
2,Afghanistan,1960,5.377778e+08
3,Africa Western and Central,1960,1.040428e+10
4,Angola,1960,NaN
...,...,...,...
16487,Kosovo,2021,NaN
16488,"Yemen, Rep.",2021,NaN
16489,South Africa,2021,NaN
16490,Zambia,2021,NaN


In [292]:
gdp_current_final['Year'] = gdp_current_final['Year'].astype("int64")

In [293]:
gdp_per_capita = pd.read_csv("/content/raw_data/gdp_per_capita.csv")

In [294]:
cols_drop = ['Country Code', 'Indicator Name', 'Indicator Code']
melts_var = [["Country Name"], "Year", "gdp_per_capita"]

In [295]:
gdp_per_capita_final = preprocessing(gdp_per_capita, cols_drop, melts_var)

In [296]:
gdp_per_capita_final

,Country Name,Year,gdp_per_capita
0,Aruba,1960,NaN
1,Africa Eastern and Southern,1960,153.494439
2,Afghanistan,1960,59.773234
3,Africa Western and Central,1960,107.932233
4,Angola,1960,NaN
...,...,...,...
16487,Kosovo,2021,NaN
16488,"Yemen, Rep.",2021,NaN
16489,South Africa,2021,NaN
16490,Zambia,2021,NaN


In [297]:
gdp_per_capita_final['Year'] = gdp_per_capita_final['Year'].astype("int64")

In [298]:
gdp_per_capita_final.dtypes

Country Name       object
Year                int64
gdp_per_capita    float64
dtype: object

In [299]:
gdp_current_final.dtypes

Country Name     object
Year              int64
gdp_current     float64
dtype: object

In [300]:
gdp_per_capita_final.rename(columns={'Country Name': 'Country'}, inplace=True)
gdp_current_final.rename(columns={'Country Name': 'Country'}, inplace=True)

### Unemployment

In [301]:
unemployment = pd.read_csv("/content/raw_data/unemployment_rate.csv")

In [302]:
cols_drop = ["Country Code", "Indicator Name", "Indicator Code"]
melts_var = [["Country Name"], "Year", "unemployment rate"]

In [303]:
unemployment_final = preprocessing(unemployment, cols_drop, melts_var)
unemployment_final

,Country Name,Year,unemployment rate
0,Aruba,1960,NaN
1,Africa Eastern and Southern,1960,NaN
2,Afghanistan,1960,NaN
3,Africa Western and Central,1960,NaN
4,Angola,1960,NaN
...,...,...,...
16487,Kosovo,2021,NaN
16488,"Yemen, Rep.",2021,13.574000
16489,South Africa,2021,33.558998
16490,Zambia,2021,13.026000


In [304]:
unemployment_final["Year"] = unemployment_final["Year"].astype("int64")

In [305]:
unemployment_final.dtypes

Country Name          object
Year                   int64
unemployment rate    float64
dtype: object

### Health Expenditure (gdp, per capita)

In [306]:
expenditure_gdp = pd.read_csv("/content/raw_data/health_expenditure_gdp.csv")

In [307]:
cols_drop = ["Country Code", "Indicator Name", "Indicator Code"]
melts_vars = [["Country Name"], "Year", "health expenditure (% GDP)"]

In [308]:
expenditure_gdp_final = preprocessing(expenditure_gdp,cols_drop,melts_vars)

In [309]:
expenditure_gdp_final.loc[expenditure_gdp_final["Country Name"] == "France"]

,Country Name,Year,health expenditure (% GDP)
77,France,1960,NaN
343,France,1961,NaN
609,France,1962,NaN
875,France,1963,NaN
1141,France,1964,NaN
...,...,...,...
15239,France,2017,11.332762
15505,France,2018,11.185830
15771,France,2019,11.057472
16037,France,2020,NaN


In [310]:
expenditure_gdp_final["Year"] = expenditure_gdp_final["Year"].astype("int64")

In [311]:
expenditure_capita = pd.read_csv("/content/raw_data/health_expenditure_per_capita.csv")


In [312]:
expenditure_capita

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Aruba,ABW,Current health expenditure per capita (current...,SH.XPD.CHEX.PC.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Africa Eastern and Southern,AFE,Current health expenditure per capita (current...,SH.XPD.CHEX.PC.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,113.183637,111.227695,104.541257,99.008506,94.739159,102.410661,96.289480,91.970113,NaN,NaN
2,Afghanistan,AFG,Current health expenditure per capita (current...,SH.XPD.CHEX.PC.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,51.440262,55.034630,59.008934,58.906528,60.188671,65.706024,69.998604,65.806030,NaN,NaN
3,Africa Western and Central,AFW,Current health expenditure per capita (current...,SH.XPD.CHEX.PC.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,71.039947,78.746863,81.804504,73.297775,63.649069,60.968846,59.328810,61.338476,NaN,NaN
4,Angola,AGO,Current health expenditure per capita (current...,SH.XPD.CHEX.PC.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,122.185585,143.606873,131.647659,108.582939,95.124977,114.334602,87.310738,71.326004,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,Kosovo,XKX,Current health expenditure per capita (current...,SH.XPD.CHEX.PC.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
262,"Yemen, Rep.",YEM,Current health expenditure per capita (current...,SH.XPD.CHEX.PC.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,75.143700,79.728119,81.051949,73.176743,NaN,NaN,NaN,NaN,NaN,NaN
263,South Africa,ZAF,Current health expenditure per capita (current...,SH.XPD.CHEX.PC.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,658.122986,604.769897,549.696167,503.759460,465.088104,534.388367,564.503296,546.688416,NaN,NaN
264,Zambia,ZMB,Current health expenditure per capita (current...,SH.XPD.CHEX.PC.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,69.295303,88.128204,67.496788,59.337643,57.362690,67.195236,76.314423,69.324379,NaN,NaN


In [314]:
cols_drop = ["Country Code", "Indicator Name", "Indicator Code"]
melts_vars = [["Country Name"], "Year", "health expenditure (per capita)"]

In [315]:
expenditure_capita_final = preprocessing(expenditure_capita, cols_drop, melts_vars)

In [316]:
expenditure_capita_final

,Country Name,Year,health expenditure (per capita)
0,Aruba,1960,NaN
1,Africa Eastern and Southern,1960,NaN
2,Afghanistan,1960,NaN
3,Africa Western and Central,1960,NaN
4,Angola,1960,NaN
...,...,...,...
16487,Kosovo,2021,NaN
16488,"Yemen, Rep.",2021,NaN
16489,South Africa,2021,NaN
16490,Zambia,2021,NaN


In [317]:
expenditure_capita_final["Year"] = expenditure_capita_final["Year"].astype("int64")

In [318]:
unemployment_final.rename(columns={'Country Name': 'Country'}, inplace=True)
expenditure_capita_final.rename(columns={'Country Name': 'Country'}, inplace=True)
expenditure_gdp_final.rename(columns={'Country Name': 'Country'}, inplace=True)

In [319]:
expenditure_capita_final.isnull().sum()

Country                                0
Year                                   0
health expenditure (per capita)    11798
dtype: int64

In [320]:
expenditure_capita_final.loc[expenditure_capita_final['Country']=='Canada']

,Country,Year,health expenditure (per capita)
35,Canada,1960,NaN
301,Canada,1961,NaN
567,Canada,1962,NaN
833,Canada,1963,NaN
1099,Canada,1964,NaN
...,...,...,...
15197,Canada,2017,4865.070801
15463,Canada,2018,5018.711426
15729,Canada,2019,5048.372559
15995,Canada,2020,NaN


### Income group

In [321]:
income = pd.read_csv("/content/raw_data/data_income.csv")

In [322]:
income.head()

,Country,Income group,Region,Lending category,1987,1988,1989,1990,1991,1992,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Aruba,High income,Latin America & Caribbean,NaN,NaN,10360.0,11760.0,12230.0,13190.0,13990.0,...,22450.0,23520.0,24510.0,25350.0,26560.0,26840.0,27120.0,NaN,NaN,NaN
1,Afghanistan,Low income,South Asia,IDA,NaN,NaN,NaN,NaN,NaN,NaN,...,530.0,630.0,660.0,630.0,600.0,550.0,530.0,520.0,530.0,500.0
2,Angola,Lower middle income,Sub-Saharan Africa,IBRD,670.0,650.0,860.0,780.0,1380.0,1170.0,...,3410.0,4170.0,4780.0,5010.0,4520.0,3770.0,3450.0,3210.0,2970.0,2230.0
3,Albania,Upper middle income,Europe & Central Asia,IBRD,730.0,730.0,760.0,650.0,410.0,280.0,...,4410.0,4360.0,4540.0,4540.0,4390.0,4320.0,4290.0,4860.0,5220.0,5210.0
4,Andorra,High income,Europe & Central Asia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [323]:
cols_drop = ["Region", "Lending category"]
melts_vars = [["Country","Income group"], "Year", "Income"]

In [324]:
income_final = preprocessing(income, cols_drop, melts_vars)

In [325]:
income_final

,Country,Income group,Year,Income
0,Aruba,High income,1987,NaN
1,Afghanistan,Low income,1987,NaN
2,Angola,Lower middle income,1987,670.0
3,Albania,Upper middle income,1987,730.0
4,Andorra,High income,1987,NaN
...,...,...,...,...
7373,Kosovo,Upper middle income,2020,4440.0
7374,"Yemen, Rep.",Low income,2020,NaN
7375,South Africa,Upper middle income,2020,5410.0
7376,Zambia,Lower middle income,2020,1190.0


In [326]:
income_final["Year"] = income_final["Year"].astype("int64")

In [327]:
income_final.dtypes

Country          object
Income group     object
Year              int64
Income          float64
dtype: object

### Second merge

In [500]:
df_merge = [gdp_current_final, gdp_per_capita_final, expenditure_capita_final,expenditure_gdp_final, unemployment_final, income_final]
left_on=['Country', 'Year']
right_on=['Country','Year']
how='left'

In [501]:
tmp_merge = merge_data(df_merge, left_on, right_on, how)

In [502]:
years_keep = list(np.arange(2000,2020))

In [503]:
tmp_merge = tmp_merge.loc[tmp_merge['Year'].isin(years_keep)]

In [504]:
tmp_merge

,Country,Year,gdp_current,gdp_per_capita,health expenditure (per capita),health expenditure (% GDP),unemployment rate,Income group,Income
10640,Aruba,2000,1.873453e+09,20617.750470,NaN,NaN,NaN,High income,20000.0
10641,Africa Eastern and Southern,2000,2.839530e+11,713.245921,44.195249,6.252428,7.724844,NaN,NaN
10642,Afghanistan,2000,NaN,NaN,NaN,NaN,10.806000,Low income,NaN
10643,Africa Western and Central,2000,1.404080e+11,525.450343,20.362792,3.771294,4.921423,NaN,NaN
10644,Angola,2000,9.129635e+09,556.838632,12.998967,1.908599,4.030000,Lower middle income,360.0
...,...,...,...,...,...,...,...,...,...
15955,Kosovo,2019,7.899879e+09,4416.108358,NaN,NaN,NaN,Upper middle income,4690.0
15956,"Yemen, Rep.",2019,NaN,NaN,NaN,NaN,13.056000,Low income,NaN
15957,South Africa,2019,3.879350e+11,6624.761865,546.688416,9.109355,28.469999,Upper middle income,6040.0
15958,Zambia,2019,2.330867e+10,1305.001031,69.324379,5.312203,12.520000,Lower middle income,1430.0


In [505]:
countries_tmp = ['Bahamas, The','Congo, Rep.','Czech Republic','Egypt, Arab Rep.','Gambia, The','Kyrgyz Republic','St. Lucia','St. Vincent and the Grenadines','Slovak Republic','Yemen, Rep.']

In [506]:
countries_replace = ['Bahamas','Congo','Czechia','Egypt','Gambia','Kyrgyzstan','Saint Lucia','Saint Vincent and the Grenadines','Slovakia','Yemen']

In [507]:
# replace country name to match with other dataset
for i, country in enumerate(countries_tmp):
  tmp_merge['Country'].replace({country:countries_replace[i]}, inplace=True)

In [508]:
tmp_merge

,Country,Year,gdp_current,gdp_per_capita,health expenditure (per capita),health expenditure (% GDP),unemployment rate,Income group,Income
10640,Aruba,2000,1.873453e+09,20617.750470,NaN,NaN,NaN,High income,20000.0
10641,Africa Eastern and Southern,2000,2.839530e+11,713.245921,44.195249,6.252428,7.724844,NaN,NaN
10642,Afghanistan,2000,NaN,NaN,NaN,NaN,10.806000,Low income,NaN
10643,Africa Western and Central,2000,1.404080e+11,525.450343,20.362792,3.771294,4.921423,NaN,NaN
10644,Angola,2000,9.129635e+09,556.838632,12.998967,1.908599,4.030000,Lower middle income,360.0
...,...,...,...,...,...,...,...,...,...
15955,Kosovo,2019,7.899879e+09,4416.108358,NaN,NaN,NaN,Upper middle income,4690.0
15956,Yemen,2019,NaN,NaN,NaN,NaN,13.056000,Low income,NaN
15957,South Africa,2019,3.879350e+11,6624.761865,546.688416,9.109355,28.469999,Upper middle income,6040.0
15958,Zambia,2019,2.330867e+10,1305.001031,69.324379,5.312203,12.520000,Lower middle income,1430.0


In [509]:
tmp_merge.isnull().sum()

Country                               0
Year                                  0
gdp_current                         230
gdp_per_capita                      230
health expenditure (per capita)     626
health expenditure (% GDP)          626
unemployment rate                   620
Income group                       1040
Income                             1514
dtype: int64

In [510]:
tmp_merge.rename(columns={'Country': 'Entity'}, inplace=True)

In [521]:
tmp_merge.to_csv("second_merge_data.csv", index=False)

## Merge primary care expenditure, disability

### Disability

In [535]:
disability = pd.read_csv("/content/raw_data/mental-and-substance-use-as-share-of-disease.csv")
disability = disability.drop(columns=['Code'], axis=1)
disability.head()

,Entity,Year,DALYs (Disability-Adjusted Life Years) - Mental disorders - Sex: Both - Age: All Ages (Percent)
0,Afghanistan,1990,1.696670
1,Afghanistan,1991,1.734281
2,Afghanistan,1992,1.791189
3,Afghanistan,1993,1.776779
4,Afghanistan,1994,1.712986


In [536]:
disability.dtypes

Entity                                                                                              object
Year                                                                                                 int64
DALYs (Disability-Adjusted Life Years) - Mental disorders - Sex: Both - Age: All Ages (Percent)    float64
dtype: object

### Final merge

In [537]:
data_frames = [first_merged, tmp_merge, disability]
left_on = ['Entity', 'Year']
right_on = ['Entity', 'Year']
how = 'left'

In [538]:
final_merge = merge_data(data_frames, left_on, right_on, how)

In [539]:
final_merge.columns

Index(['Entity', 'Year',
       'Prevalence - Mental disorders - Sex: Both - Age: Age-standardized (Percent)',
       'Deaths - Mental and substance use disorders - Sex: Both - Age: Age-standardized (Rate)',
       'Deaths - Self-harm - Sex: Both - Age: All Ages (Percent)',
       'Suicide rate', 'gdp_current', 'gdp_per_capita',
       'health expenditure (per capita)', 'health expenditure (% GDP)',
       'unemployment rate', 'Income group', 'Income',
       'DALYs (Disability-Adjusted Life Years) - Mental disorders - Sex: Both - Age: All Ages (Percent)'],
      dtype='object')

In [540]:
# reorder columns
cols_order = ['Entity', 'Year',
       'Prevalence - Mental disorders - Sex: Both - Age: Age-standardized (Percent)',
       'Deaths - Self-harm - Sex: Both - Age: All Ages (Percent)',
       'Deaths - Mental and substance use disorders - Sex: Both - Age: Age-standardized (Rate)',
       'gdp_current', 'gdp_per_capita', 'health expenditure (per capita)',
       'health expenditure (% GDP)', 'unemployment rate', 'Income group',
       'Income',
       'DALYs (Disability-Adjusted Life Years) - Mental disorders - Sex: Both - Age: All Ages (Percent)',
       'Suicide rate']

In [541]:
final_merge = final_merge[cols_order]

In [542]:
final_merge

,Entity,Year,Prevalence - Mental disorders - Sex: Both - Age: Age-standardized (Percent),Deaths - Self-harm - Sex: Both - Age: All Ages (Percent),Deaths - Mental and substance use disorders - Sex: Both - Age: Age-standardized (Rate),gdp_current,gdp_per_capita,health expenditure (per capita),health expenditure (% GDP),unemployment rate,Income group,Income,DALYs (Disability-Adjusted Life Years) - Mental disorders - Sex: Both - Age: All Ages (Percent),Suicide rate
0,Afghanistan,2000,17.939269,0.442532,0.090314,NaN,NaN,NaN,NaN,10.806,Low income,NaN,1.893882,7.7 [4.4-12.5]
1,Afghanistan,2001,17.933525,0.457531,0.084862,NaN,NaN,NaN,NaN,10.809,Low income,NaN,1.963984,7.9 [4.5-12.8]
2,Afghanistan,2002,17.916547,0.461904,0.081991,4.055180e+09,179.426579,15.803164,9.443390,11.257,Low income,NaN,2.090646,7.9 [4.5-12.8]
3,Afghanistan,2003,17.889702,0.490590,0.081942,4.515559e+09,190.683814,17.035744,8.941258,11.141,Low income,NaN,2.240707,7.7 [4.5-12.5]
4,Afghanistan,2004,17.859903,0.514944,0.082468,5.226779e+09,211.382074,20.412764,9.808474,10.988,Low income,NaN,2.360176,7.8 [4.5-12.6]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3255,Zimbabwe,2015,11.156429,1.676438,1.413464,1.996312e+10,1445.069702,107.686999,7.452066,4.778,Lower middle income,1280.0,2.193166,30.7 [16.6-48.3]
3256,Zimbabwe,2016,11.164133,1.747906,1.445058,2.054868e+10,1464.588957,112.411404,7.675163,4.788,Lower middle income,1290.0,2.279813,28.7 [15.4-45.3]
3257,Zimbabwe,2017,11.170427,1.804474,1.470020,1.758489e+10,1235.189032,115.646201,7.469752,4.785,Lower middle income,1390.0,2.364265,25.9 [13.8-40.9]
3258,Zimbabwe,2018,11.158765,1.870430,NaN,1.811554e+10,1254.642265,141.369308,8.680062,4.796,Lower middle income,1410.0,2.472949,23.9 [12.7-37.8]


In [546]:
tmp = final_merge['Suicide rate'].str.split('[', 1, expand=True)

In [547]:
tmp

,0,1
0,7.7,4.4-12.5]
1,7.9,4.5-12.8]
2,7.9,4.5-12.8]
3,7.7,4.5-12.5]
4,7.8,4.5-12.6]
...,...,...
3255,30.7,16.6-48.3]
3256,28.7,15.4-45.3]
3257,25.9,13.8-40.9]
3258,23.9,12.7-37.8]


In [548]:
final_merge['Suicide rate'] = tmp[0]

In [549]:
final_merge

,Entity,Year,Prevalence - Mental disorders - Sex: Both - Age: Age-standardized (Percent),Deaths - Self-harm - Sex: Both - Age: All Ages (Percent),Deaths - Mental and substance use disorders - Sex: Both - Age: Age-standardized (Rate),gdp_current,gdp_per_capita,health expenditure (per capita),health expenditure (% GDP),unemployment rate,Income group,Income,DALYs (Disability-Adjusted Life Years) - Mental disorders - Sex: Both - Age: All Ages (Percent),Suicide rate
0,Afghanistan,2000,17.939269,0.442532,0.090314,NaN,NaN,NaN,NaN,10.806,Low income,NaN,1.893882,7.7
1,Afghanistan,2001,17.933525,0.457531,0.084862,NaN,NaN,NaN,NaN,10.809,Low income,NaN,1.963984,7.9
2,Afghanistan,2002,17.916547,0.461904,0.081991,4.055180e+09,179.426579,15.803164,9.443390,11.257,Low income,NaN,2.090646,7.9
3,Afghanistan,2003,17.889702,0.490590,0.081942,4.515559e+09,190.683814,17.035744,8.941258,11.141,Low income,NaN,2.240707,7.7
4,Afghanistan,2004,17.859903,0.514944,0.082468,5.226779e+09,211.382074,20.412764,9.808474,10.988,Low income,NaN,2.360176,7.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3255,Zimbabwe,2015,11.156429,1.676438,1.413464,1.996312e+10,1445.069702,107.686999,7.452066,4.778,Lower middle income,1280.0,2.193166,30.7
3256,Zimbabwe,2016,11.164133,1.747906,1.445058,2.054868e+10,1464.588957,112.411404,7.675163,4.788,Lower middle income,1290.0,2.279813,28.7
3257,Zimbabwe,2017,11.170427,1.804474,1.470020,1.758489e+10,1235.189032,115.646201,7.469752,4.785,Lower middle income,1390.0,2.364265,25.9
3258,Zimbabwe,2018,11.158765,1.870430,NaN,1.811554e+10,1254.642265,141.369308,8.680062,4.796,Lower middle income,1410.0,2.472949,23.9


In [552]:
final_merge.isnull().sum()

Entity                                                                                               0
Year                                                                                                 0
Prevalence - Mental disorders - Sex: Both - Age: Age-standardized (Percent)                          0
Deaths - Self-harm - Sex: Both - Age: All Ages (Percent)                                             0
Deaths - Mental and substance use disorders - Sex: Both - Age: Age-standardized (Rate)             326
gdp_current                                                                                         37
gdp_per_capita                                                                                      37
health expenditure (per capita)                                                                     76
health expenditure (% GDP)                                                                          76
unemployment rate                                                        

In [554]:
final_merge.to_csv("./final_merge_data.csv", index=False)